# Pandas — Laboratorio 2 (CE de IA e Big Data)

**Propósito deste laboratorio:** profundizar na manipulación e análise de datos con Pandas. As **celas de código inclúen solucións**, pero **o enunciado** describe con detalle que se espera que fagas en cada paso.

**Temas:** modificación de valores, tratamento de nulos, estatística e agregacións, combinación de táboas, reestruturación (long/wide), e series temporais e datos categóricos.

Empregaremos un **dataset sintético realista** de vendas: clientes, produtos e pedidos xerados dentro do notebook para reproducibilidade.

## 0) Preparación e creación dun dataset realista
**Obxectivo:** crear tres táboas pequenas e consistentes (clientes, produtos e pedidos) que empregaremos en todo o laboratorio.

**Tarefas:**
- Fixar semente aleatoria para reproducibilidade.
- Crear `customers(customer_id, name, city, segment)` con 10 filas.
- Crear `products(product_id, category, product_name, unit_price)` con 8 filas.
- Crear `orders(order_id, date, customer_id, product_id, qty, discount)` con ~80 filas, datas entre 2024-09-01 e 2025-02-28.
- Introducir algúns nulos (en `qty` e `discount`) para practicar tratamento de valores ausentes.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(42)
pd.set_option("display.max_rows", 20)

# Táboa de clientes
customers = pd.DataFrame({
    "customer_id": range(1001, 1011),
    "name": ["Ana","Brais","Clara","Dario","Eva","Fran","Gala","Hugo","Iria","Joel"],
    "city": ["Vigo","A Coruña","Lugo","Ourense","Vigo","Santiago","Ferrol","Pontevedra","Vigo","Lugo"],
    "segment": np.random.choice(["consumer","corporate","home-office"], size=10, p=[0.6,0.25,0.15])
})

# Táboa de produtos
products = pd.DataFrame({
    "product_id": range(2001, 2009),
    "category": ["Electrónica","Electrónica","Oficina","Oficina","Hogar","Hogar","Deporte","Deporte"],
    "product_name": ["Auriculares","Teclado","Caderno","Bolígrafos","Lámpada","Tupper","Pesa 5kg","Esterilla"],
    "unit_price": [39.9, 29.5, 3.2, 1.1, 18.0, 6.5, 22.0, 15.0]
})

# Pedidos: 80 filas entre 2024-09-01 e 2025-02-28
dates = pd.date_range("2024-09-01", "2025-02-28", freq="D")
n_orders = 80
orders = pd.DataFrame({
    "order_id": range(5001, 5001+n_orders),
    "date": np.random.choice(dates, size=n_orders, replace=True),
    "customer_id": np.random.choice(customers["customer_id"], size=n_orders, replace=True),
    "product_id": np.random.choice(products["product_id"], size=n_orders, replace=True),
    "qty": np.random.randint(1, 5, size=n_orders),
    "discount": np.round(np.random.choice([0.0, 0.05, 0.10, 0.15], size=n_orders, p=[0.5,0.2,0.2,0.1]), 2)
})

# Introducimos nulos intencionados
mask = np.random.choice([True, False], size=n_orders, p=[0.1, 0.9])
orders.loc[mask, "discount"] = np.nan
mask2 = np.random.choice([True, False], size=n_orders, p=[0.05, 0.95])
orders.loc[mask2, "qty"] = np.nan

customers.head(), products.head(), orders.head()


(   customer_id   name      city      segment
 0         1001    Ana      Vigo     consumer
 1         1002  Brais  A Coruña  home-office
 2         1003  Clara      Lugo    corporate
 3         1004  Dario   Ourense     consumer
 4         1005    Eva      Vigo     consumer,
    product_id     category product_name  unit_price
 0        2001  Electrónica  Auriculares        39.9
 1        2002  Electrónica      Teclado        29.5
 2        2003      Oficina      Caderno         3.2
 3        2004      Oficina   Bolígrafos         1.1
 4        2005        Hogar      Lámpada        18.0,
    order_id       date  customer_id  product_id  qty  discount
 0      5001 2025-01-28         1003        2002  3.0      0.10
 1      5002 2024-10-23         1001        2004  4.0      0.00
 2      5003 2024-09-02         1008        2004  1.0      0.05
 3      5004 2024-11-27         1003        2006  1.0      0.05
 4      5005 2025-02-05         1003        2007  NaN      0.05)

## 1) Modificación de valores e novas columnas
**Obxectivo:** preparar unha táboa de pedidos enriquecida con información de prezo e métricas calculadas.

**Tarefas detalladas:**
1. **Enriquecer prezos:** facer `merge` entre `orders` e `products` (trátese só de traer `unit_price`).
2. **Calendario:** engadir `year` e `month` extraídos de `date`.
3. **Subtotais:** crear `subtotal = qty * unit_price` (ten en conta que `qty` pode ter nulos).
4. **Descontos:** substituír nulos en `discount` por 0 nunha nova columna `discount_filled`.
5. **Total:** calcular `total = subtotal * (1 - discount_filled)`.
6. **Verificación rápida:** amosar `head()` para comprobar resultados.

In [3]:
# Merge para prezos e creación de novas columnas
orders_enriched = orders.merge(products[["product_id","unit_price"]], on="product_id", how="left")
orders_enriched["year"] = pd.to_datetime(orders_enriched["date"]).dt.year
orders_enriched["month"] = pd.to_datetime(orders_enriched["date"]).dt.month
orders_enriched["subtotal"] = orders_enriched["qty"] * orders_enriched["unit_price"]
orders_enriched["discount_filled"] = orders_enriched["discount"].fillna(0.0)
orders_enriched["total"] = orders_enriched["subtotal"] * (1 - orders_enriched["discount_filled"])
orders_enriched.head()


,order_id,date,customer_id,product_id,qty,discount,unit_price,year,month,subtotal,discount_filled,total
0,5001,2025-01-28,1003,2002,3.0,0.10,29.5,2025,1,88.5,0.10,79.650
1,5002,2024-10-23,1001,2004,4.0,0.00,1.1,2024,10,4.4,0.00,4.400
2,5003,2024-09-02,1008,2004,1.0,0.05,1.1,2024,9,1.1,0.05,1.045
3,5004,2024-11-27,1003,2006,1.0,0.05,6.5,2024,11,6.5,0.05,6.175
4,5005,2025-02-05,1003,2007,NaN,0.05,22.0,2025,2,NaN,0.05,NaN


## 2) Tratamento de valores ausentes
**Obxectivo:** identificar, contar e tratar valores ausentes segundo diferentes necesidades.

**Tarefas detalladas:**
1. **Conteo de nulos por columna:** usar `isna().sum()` e visualizar só as columnas con nulos.
2. **Estratexia 1 – eliminación:** crear `orders_dropna` eliminando filas con nulos en `qty` **ou** `discount`.
3. **Estratexia 2 – substitución:** crear `orders_fillna` onde `qty` se cobre con 1 e `discount` con 0.0.
4. **Comparación:** revisar `head()` de cada versión e reflexionar sobre cando usar cada estratexia.

In [6]:
# 2.1) Conteo de nulos
null_counts = orders_enriched.isna().sum()
null_counts[null_counts > 0]


qty          1
discount    11
subtotal     1
total        1
dtype: int64

In [7]:
# 2.2) Eliminación fronte a substitución
orders_dropna = orders_enriched.dropna(subset=["qty","discount"])

orders_fillna = orders_enriched.copy()
orders_fillna["qty"] = orders_fillna["qty"].fillna(1)
orders_fillna["discount"] = orders_fillna["discount"].fillna(0.0)

orders_dropna.head(), orders_fillna.head()


(   order_id       date  customer_id  product_id  qty  discount  unit_price  \
 0      5001 2025-01-28         1003        2002  3.0      0.10        29.5   
 1      5002 2024-10-23         1001        2004  4.0      0.00         1.1   
 2      5003 2024-09-02         1008        2004  1.0      0.05         1.1   
 3      5004 2024-11-27         1003        2006  1.0      0.05         6.5   
 5      5006 2024-10-08         1001        2002  3.0      0.00        29.5   
 
    year  month  subtotal  discount_filled   total  
 0  2025      1      88.5             0.10  79.650  
 1  2024     10       4.4             0.00   4.400  
 2  2024      9       1.1             0.05   1.045  
 3  2024     11       6.5             0.05   6.175  
 5  2024     10      88.5             0.00  88.500  ,
    order_id       date  customer_id  product_id  qty  discount  unit_price  \
 0      5001 2025-01-28         1003        2002  3.0      0.10        29.5   
 1      5002 2024-10-23         1001        200

## 3) Operacións estatísticas e agregacións
**Obxectivo:** resumir e obter métricas útiles.

**Tarefas detalladas:**
1. `describe()` sobre `qty`, `unit_price`, `subtotal`, `total` para ter un resumo estatístico.
2. `value_counts()` de `city` (clientes) e `category` (produtos) para coñecer distribucións.
3. **Por categoría:** calcular **ingreso total** e **desconto medio**.
4. **Por mes:** usar `groupby('month').agg(...)` para obter `total_sum`, `qty_mean`, `discount_mean` e `orders_count`.
5. **Describe por grupo:** combinar `groupby('category')` con `describe()` sobre `total`.
6. **Correlación:** matriz de correlación entre `qty`, `unit_price`, `subtotal` e `total`.

In [8]:
# 3.1) describe()
orders_enriched[["qty","unit_price","subtotal","total"]].describe()


,qty,unit_price,subtotal,total
count,79.000000,80.000000,79.000000,79.000000
mean,2.708861,16.636250,44.843038,43.304684
std,1.210551,13.393549,43.007831,42.026537
min,1.000000,1.100000,1.100000,1.045000
25%,2.000000,3.200000,6.500000,6.450000
50%,3.000000,15.000000,29.500000,26.000000
75%,4.000000,29.500000,79.800000,73.905000
max,4.000000,39.900000,159.600000,159.600000


In [9]:
# 3.2) value_counts por city e category
vc_city = customers["city"].value_counts()
vc_cat = products["category"].value_counts()
vc_city, vc_cat


(city
 Vigo          3
 Lugo          2
 A Coruña      1
 Ourense       1
 Santiago      1
 Ferrol        1
 Pontevedra    1
 Name: count, dtype: int64,
 category
 Electrónica    2
 Oficina        2
 Hogar          2
 Deporte        2
 Name: count, dtype: int64)

In [10]:
# 3.3) ingreso total e desconto medio por categoría
tmp = orders_enriched.merge(products[["product_id","category"]], on="product_id", how="left")
agg_cat = tmp.groupby("category").agg(total_ingreso=("total","sum"),
                                      desconto_medio=("discount_filled","mean"))
agg_cat


,total_ingreso,desconto_medio
category,,
Deporte,1041.900,0.045455
Electrónica,1891.715,0.038095
Hogar,353.925,0.019231
Oficina,133.530,0.033333


In [11]:
# 3.4) agregación por mes
agg_mes = orders_enriched.groupby("month").agg(
    total_sum=("total","sum"),
    qty_mean=("qty","mean"),
    discount_mean=("discount_filled","mean"),
    orders_count=("order_id","count")
).sort_index()
agg_mes


,total_sum,qty_mean,discount_mean,orders_count
month,,,,
1,595.365,3.000000,0.037500,12
2,454.650,2.600000,0.022727,11
9,680.220,2.562500,0.037500,16
10,772.005,2.900000,0.027500,20
11,664.100,2.500000,0.035714,14
12,254.730,2.571429,0.071429,7


In [12]:
# 3.5) describe por grupo
gb_desc = tmp.groupby("category")["total"].describe()
gb_desc


,count,mean,std,min,25%,50%,75%,max
category,,,,,,,,
Deporte,21.0,49.614286,25.948710,12.750,22.00000,57.000,66.00,88.00
Electrónica,21.0,90.081667,40.980125,25.075,67.83000,84.075,118.00,159.60
Hogar,13.0,27.225000,22.518613,5.850,6.50000,23.400,36.00,72.00
Oficina,24.0,5.563750,3.578917,1.045,3.18375,4.400,7.08,12.16


In [13]:
# 3.6) correlación
orders_enriched[["qty","unit_price","subtotal","total"]].corr()


,qty,unit_price,subtotal,total
qty,1.000000,-0.002381,0.428028,0.430318
unit_price,-0.002381,1.000000,0.837901,0.827961
subtotal,0.428028,0.837901,1.000000,0.997032
total,0.430318,0.827961,0.997032,1.000000


## 4) Combinación de datos de varias táboas (merge, concat)
**Obxectivo:** unir información entre táboas e combinar rexistros.

**Tarefas detalladas:**
1. **Merge con customers:** enriquecer `orders_enriched` con `city` e `segment`. Logo calcular o **ingreso medio por cidade**.
2. **Concat de novos pedidos:** crear unha pequena mostra `orders_new` (5 filas) e concatenala con `orders_enriched`; asegurarse de eliminar duplicados por `order_id`.

In [14]:
# 4.1) MERGE con customers e ingreso medio por cidade
orders_full = orders_enriched.merge(customers[["customer_id","city","segment"]], on="customer_id", how="left")
ingreso_medio_city = orders_full.groupby("city")["total"].mean().sort_values(ascending=False)
orders_full.head(), ingreso_medio_city.head()


(   order_id       date  customer_id  product_id  qty  discount  unit_price  \
 0      5001 2025-01-28         1003        2002  3.0      0.10        29.5   
 1      5002 2024-10-23         1001        2004  4.0      0.00         1.1   
 2      5003 2024-09-02         1008        2004  1.0      0.05         1.1   
 3      5004 2024-11-27         1003        2006  1.0      0.05         6.5   
 4      5005 2025-02-05         1003        2007  NaN      0.05        22.0   
 
    year  month  subtotal  discount_filled   total        city      segment  
 0  2025      1      88.5             0.10  79.650        Lugo    corporate  
 1  2024     10       4.4             0.00   4.400        Vigo     consumer  
 2  2024      9       1.1             0.05   1.045  Pontevedra  home-office  
 3  2024     11       6.5             0.05   6.175        Lugo    corporate  
 4  2025      2       NaN             0.05     NaN        Lugo    corporate  ,
 city
 A Coruña      98.600000
 Santiago      64.960000

In [15]:
# 4.2) CONCAT con pedidos novos e eliminación de duplicados
orders_new = orders_enriched.sample(5, random_state=1).copy()
orders_new["order_id"] = orders_new["order_id"] + 10000
orders_all = pd.concat([orders_enriched, orders_new], ignore_index=True).drop_duplicates(subset=["order_id"])
len(orders_enriched), len(orders_all)


(80, 85)

## 5) Reestruturación (long/wide, pivot, stack/unstack)
**Obxectivo:** cambiar a forma dos datos para análises alternativas.

**Tarefas detalladas:**
1. **Formato long con `melt`:** transformar `qty` e `total` en filas, mantendo `month` e `category` como id.
2. **Pivot table:** obter unha táboa co **índice** `month`, **columnas** `category` e **valores** a suma de `total`.
3. **Stack / Unstack:** practicar a ida e volta a partir do pivot anterior.

In [16]:
# 5.1) melt (wide -> long)
tmp2 = orders_enriched.merge(products[["product_id","category"]], on="product_id", how="left")
long_df = pd.melt(tmp2, id_vars=["month","category"], value_vars=["qty","total"],
                  var_name="metric", value_name="value")
long_df.head()


,month,category,metric,value
0,1,Electrónica,qty,3.0
1,10,Oficina,qty,4.0
2,9,Oficina,qty,1.0
3,11,Hogar,qty,1.0
4,2,Deporte,qty,NaN


In [17]:
# 5.2) pivot table (long -> wide)
pivot_df = tmp2.pivot_table(index="month", columns="category", values="total", aggfunc="sum")
pivot_df


category,Deporte,Electrónica,Hogar,Oficina
month,,,,
1,111.00,441.325,NaN,43.04
2,194.75,147.500,108.000,4.40
9,374.40,256.930,30.350,18.54
10,118.00,434.355,186.000,33.65
11,223.95,424.075,6.175,9.90
12,19.80,187.530,23.400,24.00


In [18]:
# 5.3) stack/unstack
stacked = pivot_df.stack()
unstacked = stacked.unstack()
stacked.head(), unstacked.head()


(month  category   
 1      Deporte        111.000
        Electrónica    441.325
        Oficina         43.040
 2      Deporte        194.750
        Electrónica    147.500
 dtype: float64,
 category  Deporte  Electrónica    Hogar  Oficina
 month                                           
 1          111.00      441.325      NaN    43.04
 2          194.75      147.500  108.000     4.40
 9          374.40      256.930   30.350    18.54
 10         118.00      434.355  186.000    33.65
 11         223.95      424.075    6.175     9.90)

## 6) Series temporais e datos categóricos
**Obxectivo:** traballar con datas e optimizar variables categóricas.

**Tarefas detalladas:**
1. **Mensualización:** converter `date` en índice temporal ordenado e facer `resample('M').sum()` de `total`.
2. **Categóricos:** converter `category` e `segment` a tipo `category` e comparar consumo de memoria **antes** e **despois**.
3. **Gráfica mensual:** debuxar a serie mensual de ingresos nun **único gráfico** con títulos e eixes.

In [19]:
# 6.1) Resample mensual
orders_ts = orders_enriched.set_index(pd.to_datetime(orders_enriched["date"])).sort_index()
mensual = orders_ts["total"].resample("M").sum()
mensual.head()


date
2024-09-30    680.220
2024-10-31    772.005
2024-11-30    664.100
2024-12-31    254.730
2025-01-31    595.365
Freq: M, Name: total, dtype: float64

In [20]:
# 6.2) Conversión a categóricos e memoria
before = tmp2[["category"]].memory_usage(deep=True).sum()
tmp2["category"] = tmp2["category"].astype("category")
orders_full["segment"] = orders_full["segment"].astype("category")
after = tmp2[["category"]].memory_usage(deep=True).sum()
before, after


(5279, 639)

In [ ]:
# 6.3) Visualización mensual
mensual.plot(figsize=(8,3), title="Ingresos mensuais")
plt.xlabel("Mes")
plt.ylabel("Ingresos")
plt.show()
